In [ ]:
%matplotlib widget

from ogdf_python import *

cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")
null_node = cppyy.bind_object(cppyy.nullptr, 'ogdf::NodeElement')

G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.randomPlanarTriconnectedGraph(G, 10, 20)
GA = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)
GA.directed = True

SL = ogdf.SugiyamaLayout()
SL.call(GA)
GA.rotateRight90()
GA

In [ ]:
# This is the main DFS code. Compare it with the slides from the lecture!

def dfs(G):
    # NodeArrays are used to store information "labelling" individual nodes
    discovery = ogdf.NodeArray[int](G, -1)
    finish = ogdf.NodeArray[int](G, -1)
    predecessor = ogdf.NodeArray["ogdf::node"](G, null_node)
    
    time = 0
    
    def dfs_visit(u):
        nonlocal time # (we need to overwrite this variable from the parent function)

        time += 1
        discovery[u] = time
        # yield stops the execution of our method and passes the variables to our caller
        yield u, discovery, finish, predecessor
        # the code will continue here the next time `next(it)` is called

        for adj in u.adjEntries:
            v = adj.twinNode()
            if adj.isSource() and discovery[v] < 0:
                predecessor[v] = u
                # yield from simply "copies over" all yield statements from the called method
                yield from dfs_visit(v)

        time += 1
        finish[u] = time
        # yield again to report the state after
        yield u, discovery, finish, predecessor
    
    for node in G.nodes:
        if discovery[node] < 0:
            yield from dfs_visit(node)

In [ ]:
def goto(steps):
    it = dfs(G)
    try:
        for _ in range(steps):
            last, discovery, finish, predecessor = next(it)
    except StopIteration:
        pass
    
    if steps < 1:
        for u in G.nodes:
            GA.label[u] = "(-1, -1)"
            GA.width[u] = 40
            GA.fillColor[u] = ogdf.Color(255,255,255)
            GA.strokeColor[u] = ogdf.Color(230, 230, 230)
        for e in G.edges:
            GA.strokeColor[e] = ogdf.Color(150, 150, 150)
        return
    
    for u in G.nodes:
        d = discovery[u]
        f = finish[u]
        GA.label[u] = "(%s, %s)" % (d, f)
        GA.width[u] = 40
        GA.fillColor[u] = ogdf.Color(255,255,255)
    
        if d < 0:
            GA.strokeColor[u] = ogdf.Color(230, 230, 230)
        elif f < 0:
            GA.strokeColor[u] = ogdf.Color(150, 150, 150)
        else:
            GA.strokeColor[u] = ogdf.Color(0, 0, 0)
    
    GA.fillColor[last] = ogdf.Color(ogdf.Color.Name.Lightpink)
    
    for e in G.edges:
        ds = discovery[e.source()]
        fs = finish[e.source()]
        dt = discovery[e.target()]
        ft = finish[e.target()]
    
        if ds < 0:
            GA.strokeColor[e] = ogdf.Color(150, 150, 150)
        elif predecessor[e.target()] == e.source():
            GA.strokeColor[e] = ogdf.Color(ogdf.Color.Name.Darkblue)
        elif dt >= 0 and dt < ds:
            if ft >= 0 and fs > ft:
                GA.strokeColor[e] = ogdf.Color(ogdf.Color.Name.Pink) # FIXME
            else:
                GA.strokeColor[e] = ogdf.Color(ogdf.Color.Name.Lightgreen)
        elif ds < dt:
            GA.strokeColor[e] = ogdf.Color(ogdf.Color.Name.Lightblue)

    return GA

In [ ]:
from ipywidgets import *
from ogdf_python.matplotlib import MatplotlibGraph

goto(0)
w = MatplotlibGraph(GA)
conf = dict(
    value=0,
    min=0,
    max=G.numberOfNodes() * 2,
    step=1)
play = widgets.Play(**conf, interval=1000)
slider = widgets.IntSlider(**conf)
widgets.jslink((play, 'value'), (slider, 'value'))
button = widgets.Button(
    description='New Graph'
)


def on_value_change(change):
    goto(slider.value)
    for na in w.nodes.values():
        na.update_attributes(GA)
    for ea in w.edges.values():
        ea.update_attributes(GA)

slider.observe(on_value_change, names='value')

display(VBox([
    HBox([play, slider, button]),
    w.ax.figure.canvas
]))

In [ ]:
display(
MatplotlibGraph(GA)\
     .ax.figure.canvas
)

In [ ]:
import os
os.getpid()